# <div align="center" style="color: #ff5733;">Finding whether the companies provide Pay day loans or not</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from google.cloud import bigquery
import os
# path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
# client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import chardet

def search_google(query):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    url = f"https://www.google.com/search?q={query}"
    response = requests.get(url, headers=headers)
    return response.text

def check_payday_loans(company_name):
    search_query = f"{company_name} payday loans"
    html_content = search_google(search_query)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    keywords = ['payday loan', 'quick loan', 'salary']
    text = soup.get_text().lower()
    
    for keyword in keywords:
        if keyword in text:
            return True
    return False

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    return chardet.detect(raw_data)['encoding']

def process_companies(input_file, output_file):
    encoding = detect_encoding(input_file)
    print(f"Detected encoding: {encoding}")

    with open(input_file, 'r', encoding=encoding) as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        writer.writerow(['Company Name', 'Registration Number', 'Trade Names', 'Provides Payday Loans'])
        
        current_row = []
        for row in reader:
            if row and row[0]:  # Start of a new entry
                if current_row:  # Process the previous entry if it exists
                    process_entry(current_row, writer)
                current_row = row
            else:  # Continuation of the previous entry
                current_row.extend(row)
        
        # Process the last entry
        if current_row:
            process_entry(current_row, writer)

def process_entry(row, writer):
    if len(row) < 3:
        print(f"Skipping invalid row: {row}")
        return
    
    company_name = row[0].strip()
    registration_number = row[1].strip()
    trade_names = ' '.join(row[2:]).strip()
    
    print(f"Checking: {company_name}")
    provides_payday_loans = check_payday_loans(company_name)
    
    writer.writerow([company_name, registration_number, trade_names, provides_payday_loans])
    
    # Sleep to avoid overwhelming the server
    time.sleep(2)

if __name__ == "__main__":
    input_file = "companies.csv"
    output_file = "payday_loan_results_seconditeration_v1.csv"
    process_companies(input_file, output_file)

Detected encoding: Windows-1252
Checking: Company Name
Checking: Acom Consumer Finance Corporation
Checking: Advance Tech Lending Inc.
Checking: Aeon Credit Service (Philippines) Inc
Checking: And Financing Corporation
Checking: Armorak Lending Inc
Checking: Asialink Finance Corporation
Checking: Asiasource Financial Inc
Checking: “Asteria Lending Inc. Inc. Doing Business under the
Checking: name and style of Cashbazaar”
Checking: Buenavista Financing Corporation
Checking: Cash Mart Asia Lending Inc.
Checking: Cashbee Lending Services Inc.
Checking: Cashmum Lending Investors Corp.
Checking: Cashxpress South East Asia Lending Inc.
Checking: Ceimmarj Financing Incorporated
Checking: Cepat Kredit Financing Inc.
Checking: Codeblock Lending Inc.
Checking: Convenience Cash Lending Corp.
Checking: Corpfund Lending Investor Corporation
Checking: Crawfort Micro Lending Technologies Philippines Inc.
Checking: Creditable Lending Corporation
Checking: Df Credit Lending Corporation
Checking: Digido

In [4]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import chardet
import re

def search_google(query):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    url = f"https://www.google.com/search?q={query}"
    response = requests.get(url, headers=headers)
    return response.text

def check_payday_loans(company_name):
    search_query = f"{company_name} loan"
    html_content = search_google(search_query)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    text = soup.get_text().lower()
    
    # Define more specific payday loan indicators
    payday_indicators = [
        r'\bpayday loan',
        r'\bshort-term loan',
        r'\bquick cash',
        r'\bquick loan',
        r'\bfast cash',
        r'\binstant loan',
        r'\bcash advance',
        r'\bsalary loan',
        r'loan.*?24 hours',
        r'loan.*?same day',
        r'loan.*?salary',
        r'no credit check.*?loan',
        r'high interest.*?loan',
        r'easy approval.*?loan'
    ]
    
    # Define exclusion terms
    exclusion_terms = [
        'against payday loans',
        'alternatives to payday loans',
        'payday loan debt',
        'dangers of payday loans'
    ]
    
    # Check for payday loan indicators
    indicator_count = sum(1 for indicator in payday_indicators if re.search(indicator, text))
    
    # Check for exclusion terms
    exclusion_count = sum(1 for term in exclusion_terms if term in text)
    
    # Determine if it's likely a payday lender
    is_payday_lender = indicator_count > 2 and exclusion_count == 0
    
    return is_payday_lender

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    return chardet.detect(raw_data)['encoding']

def process_companies(input_file, output_file):
    encoding = detect_encoding(input_file)
    print(f"Detected encoding: {encoding}")

    with open(input_file, 'r', encoding=encoding) as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        writer.writerow(['Company Name', 'Registration Number', 'Trade Names', 'Likely Provides Payday Loans'])
        
        current_row = []
        for row in reader:
            if row and row[0]:  # Start of a new entry
                if current_row:  # Process the previous entry if it exists
                    process_entry(current_row, writer)
                current_row = row
            else:  # Continuation of the previous entry
                current_row.extend(row)
        
        # Process the last entry
        if current_row:
            process_entry(current_row, writer)

def process_entry(row, writer):
    if len(row) < 3:
        print(f"Skipping invalid row: {row}")
        return
    
    company_name = row[0].strip()
    registration_number = row[1].strip()
    trade_names = ' '.join(row[2:]).strip()
    
    print(f"Checking: {company_name}")
    likely_provides_payday_loans = check_payday_loans(company_name)
    
    writer.writerow([company_name, registration_number, trade_names, likely_provides_payday_loans])
    
    # Sleep to avoid overwhelming the server
    time.sleep(2)

if __name__ == "__main__":
    input_file = "companies.csv"
    output_file = "payday_loan_results_thirditeration_v2.csv"
    process_companies(input_file, output_file)

Detected encoding: Windows-1252
Checking: Company Name
Checking: Acom Consumer Finance Corporation
Checking: Advance Tech Lending Inc.
Checking: Aeon Credit Service (Philippines) Inc
Checking: And Financing Corporation
Checking: Armorak Lending Inc
Checking: Asialink Finance Corporation
Checking: Asiasource Financial Inc
Checking: “Asteria Lending Inc. Inc. Doing Business under the
Checking: name and style of Cashbazaar”
Checking: Buenavista Financing Corporation
Checking: Cash Mart Asia Lending Inc.
Checking: Cashbee Lending Services Inc.
Checking: Cashmum Lending Investors Corp.
Checking: Cashxpress South East Asia Lending Inc.
Checking: Ceimmarj Financing Incorporated
Checking: Cepat Kredit Financing Inc.
Checking: Codeblock Lending Inc.
Checking: Convenience Cash Lending Corp.
Checking: Corpfund Lending Investor Corporation
Checking: Crawfort Micro Lending Technologies Philippines Inc.
Checking: Creditable Lending Corporation
Checking: Df Credit Lending Corporation
Checking: Digido

In [5]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import chardet
import re

def search_google(query):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    url = f"https://www.google.com/search?q={query}"
    response = requests.get(url, headers=headers)
    return response.text

def check_payday_loans(company_name):
    search_query = f"{company_name} loan"
    html_content = search_google(search_query)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    text = soup.get_text().lower()
    
    # Define more specific payday loan indicators
    payday_indicators = [
        r'\bpayday loan',
        r'\bshort-term loan',
        r'\bquick cash',
        r'\bquick loan',
        r'\bfast cash',
        r'\binstant loan',
        r'\bcash advance',
        r'\bsalary loan',
        r'loan.*?24 hours',
        r'loan.*?same day',
        r'loan.*?salary',
        r'no credit check.*?loan',
        r'high interest.*?loan',
        r'easy approval.*?loan'
    ]
    
    # Define exclusion terms
    exclusion_terms = [
        'against payday loans',
        'alternatives to payday loans',
        'payday loan debt',
        'dangers of payday loans'
    ]
    
    # Check for payday loan indicators
    matching_indicators = [indicator for indicator in payday_indicators if re.search(indicator, text)]
    indicator_count = len(matching_indicators)
    
    # Check for exclusion terms
    exclusion_count = sum(1 for term in exclusion_terms if term in text)
    
    # Determine if it's likely a payday lender
    is_payday_lender = indicator_count > 2 and exclusion_count == 0
    
    return is_payday_lender, matching_indicators, indicator_count

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    return chardet.detect(raw_data)['encoding']

def process_companies(input_file, output_file):
    encoding = detect_encoding(input_file)
    print(f"Detected encoding: {encoding}")

    with open(input_file, 'r', encoding=encoding) as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        writer.writerow(['Company Name', 'Registration Number', 'Trade Names', 'Likely Provides Payday Loans', 'Matching Keywords', 'Payday Indicators Count'])
        
        current_row = []
        for row in reader:
            if row and row[0]:  # Start of a new entry
                if current_row:  # Process the previous entry if it exists
                    process_entry(current_row, writer)
                current_row = row
            else:  # Continuation of the previous entry
                current_row.extend(row)
        
        # Process the last entry
        if current_row:
            process_entry(current_row, writer)

def process_entry(row, writer):
    if len(row) < 3:
        print(f"Skipping invalid row: {row}")
        return
    
    company_name = row[0].strip()
    registration_number = row[1].strip()
    trade_names = ' '.join(row[2:]).strip()
    
    print(f"Checking: {company_name}")
    likely_provides_payday_loans, matching_keywords, indicator_count = check_payday_loans(company_name)
    
    writer.writerow([
        company_name,
        registration_number,
        trade_names,
        likely_provides_payday_loans,
        ', '.join(matching_keywords),
        indicator_count
    ])
    
    # Sleep to avoid overwhelming the server
    time.sleep(2)

if __name__ == "__main__":
    input_file = "companies.csv"
    output_file = "payday_loan_results_detailed.csv"
    process_companies(input_file, output_file)

Detected encoding: Windows-1252
Checking: Company Name
Checking: Acom Consumer Finance Corporation
Checking: Advance Tech Lending Inc.
Checking: Aeon Credit Service (Philippines) Inc
Checking: And Financing Corporation
Checking: Armorak Lending Inc
Checking: Asialink Finance Corporation
Checking: Asiasource Financial Inc
Checking: “Asteria Lending Inc. Inc. Doing Business under the
Checking: name and style of Cashbazaar”
Checking: Buenavista Financing Corporation
Checking: Cash Mart Asia Lending Inc.
Checking: Cashbee Lending Services Inc.
Checking: Cashmum Lending Investors Corp.
Checking: Cashxpress South East Asia Lending Inc.
Checking: Ceimmarj Financing Incorporated
Checking: Cepat Kredit Financing Inc.
Checking: Codeblock Lending Inc.
Checking: Convenience Cash Lending Corp.
Checking: Corpfund Lending Investor Corporation
Checking: Crawfort Micro Lending Technologies Philippines Inc.
Checking: Creditable Lending Corporation
Checking: Df Credit Lending Corporation
Checking: Digido